In [61]:
import os
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

from matplotlib import pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from tqdm import tqdm

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin, space_eval

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
df = pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
input_shape = X_train[0].shape
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, y_train,
            batch_size=params_fit.get('batch_size', 128),
            epochs=params_fit.get('epochs', 5),
            verbose=params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', (X_test, y_test)),
            callbacks=[tensorboard_callback]
            )
  
  return model


def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)



def train_and_predict(model, params):
  model_trained = train_model(model, X_train, y_train, params_fit=params)
  return predict(model_trained, X_test, y_test)


In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
    ])

In [13]:
train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.1270 - accuracy: 0.4055 - val_loss: 0.5168 - val_accuracy: 0.8449
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 0.5305 - accuracy: 0.8324 - val_loss: 0.1326 - val_accuracy: 0.9590
Epoch 3/5
272/272 [==============================] - 4s 16ms/step - loss: 0.2799 - accuracy: 0.9159 - val_loss: 0.0664 - val_accuracy: 0.9821
Epoch 4/5
272/272 [==============================] - 4s 16ms/step - loss: 0.1974 - accuracy: 0.9410 - val_loss: 0.0471 - val_accuracy: 0.9868
Epoch 5/5
272/272 [==============================] - 4s 16ms/step - loss: 0.1593 - accuracy: 0.9545 - val_loss: 0.0362 - val_accuracy: 0.9901


0.9712018140589569

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),
    Dense(num_classes, activation='softmax')
    ])

In [0]:
def func_obj(params):

  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])

  model.fit(X_train, y_train,
            batch_size=params.get('batch_size', 128),
            epochs=5,
            verbose=0
            )
  
  score = model.evaluate(X_test, y_test)
  print(f'accuracy={score[1]}')

  return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [63]:
space = {
    'batch_size': hp.choice('batch_size', [16, 32, 64, 128, 256, 512]),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0, 1),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0, 1),
}

best = fmin(
    fn=func_obj,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 2.8461 - accuracy: 0.0000e+00

 11/138 [=>............................]
 - ETA: 0s - loss: 2.5074 - accuracy: 0.0938    

 19/138 [===>..........................]
 - ETA: 0s - loss: 2.6450 - accuracy: 0.1250

 28/138 [=====>........................]
 - ETA: 0s - loss: 2.7389 - accuracy: 0.0949

 36/138 [======>.......................]
 - ETA: 0s - loss: 2.5694 - accuracy: 0.1398

 45/138 [========>.....................]
 - ETA: 0s - loss: 2.7002 - accuracy: 0.1153


In [64]:
space_eval(space, best)

{'batch_size': 128,
 'dropout_cnn_block_one': 0.05474733089123855,
 'dropout_cnn_block_three': 0.37493622854285574,
 'dropout_cnn_block_two': 0.059166985150330906,
 'dropout_dense_block_one': 0.04927739048100532,
 'dropout_dense_block_two': 0.853071273847178}

In [71]:
train_and_predict(model=get_model(space_eval(space, best)), params=space_eval(space, best))

Epoch 1/5
272/272 [==============================] - 4s 13ms/step - loss: 2.5826 - accuracy: 0.2870 - val_loss: 1.3143 - val_accuracy: 0.5823
Epoch 2/5
272/272 [==============================] - 3s 12ms/step - loss: 0.9096 - accuracy: 0.7194 - val_loss: 0.3442 - val_accuracy: 0.8984
Epoch 3/5
272/272 [==============================] - 3s 12ms/step - loss: 0.3662 - accuracy: 0.8954 - val_loss: 0.1593 - val_accuracy: 0.9544
Epoch 4/5
272/272 [==============================] - 3s 12ms/step - loss: 0.2113 - accuracy: 0.9429 - val_loss: 0.1304 - val_accuracy: 0.9664
Epoch 5/5
272/272 [==============================] - 3s 12ms/step - loss: 0.1564 - accuracy: 0.9592 - val_loss: 0.1109 - val_accuracy: 0.9696


0.9696145124716553